In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [1]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_3_2018_territorial_scope_after_public_consultation_en_1.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/territorial_scope.md"

In [4]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/territorial_scope.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'([a-h])\)\s*(.*)') 
# subpoint_pattern = re.compile(r'^(\d+)\.\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = section + "." + subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0, 1, 2])
df.reset_index(drop=True, inplace=True)
df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
df.at[0, "heading"] = True
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[180:200]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,section,subsection,point,heading,text,section_reference
180,,,,False,"As per Article 30, the controller or processor...",4.d
181,,,,False,"As clarified by recital 80, the representative...",4.d
182,,,,False,"With the help of a team if necessary, the repr...",4.d
183,,,,False,It should however be noted that the concept of...,4.d
184,,,,False,The EDPB furthermore highlights that article 5...,4.d


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 15 augmented with footnotes
Row 16 augmented with footnotes
Row 17 augmented with footnotes
Row 18 augmented with footnotes
Row 19 augmented with footnotes
Row 21 augmented with footnotes
Row 24 augmented with footnotes
Row 27 augmented with footnotes
Row 29 augmented with footnotes
Row 44 augmented with footnotes
Row 57 augmented with footnotes
Row 69 augmented with footnotes
Row 72 augmented with footnotes
Row 81 augmented with footnotes
Row 94 augmented with footnotes
Row 95 augmented with footnotes
Row 99 augmented with footnotes
Row 100 augmented with footnotes
Row 149 augmented with footnotes
Row 159 augmented with footnotes
Row 160 augmented with footnotes
Row 165 augmented with footnotes
Row 168 augmented with footnotes
Row 169 augmented with footnotes
Row 170 augmented with footnotes
Row 173 augmented with footnotes


In [6]:
file = "../../inputs/documents/territorial_scope.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [10]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.territorial_scope
importlib.reload(gdpr_rag.documents.territorial_scope)
from gdpr_rag.documents.territorial_scope import TerritorialScope

path_to_manual_as_csv_file = "../../inputs/documents/territorial_scope.parquet"

doc = TerritorialScope(path_to_manual_as_csv_file)


In [17]:
from IPython.display import Markdown, display

section = "1"

section = "4.a"


print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


4 REPRESENTATIVE OF CONTROLLERS OR PROCESSORS NOT  ESTABLISHED IN THE UNION
4.a Designation of a representative


# 4 REPRESENTATIVE OF CONTROLLERS OR PROCESSORS NOT  ESTABLISHED IN THE UNION

## 4.a Designation of a representative

Recital 80 clarifies that "[t]he representative should be explicitly designated by a written mandate of  the controller or of the processor to act on its behalf with regard to its obligations under this Regulation.  The designation of such a representative does not affect the responsibility or liability of the controller  or of the processor under this Regulation. Such a representative should perform its tasks according to  the mandate received from the controller or processor, including cooperating with the competent  supervisory authorities with regard to any action taken to ensure compliance with this Regulation."

The written mandate referred to in Recital 80 shall therefore govern the relations and obligations  between the representative in the Union and the data controller or processor established outside the  Union, while not affecting the responsibility or liability of the controller or processor. The  representative in the Union may be a natural or a legal person established in the Union able to  represent a data controller or processor established outside the Union with regard to their respective  obligations under the GDPR.

In practice, the function of representative in the Union can be exercised based on a service contract  concluded with an individual or an organisation, and can therefore be assumed by a wide range of  commercial and non-commercial entities, such as law firms, consultancies, private companies, etc…  provided that such entities are established in the Union. One representative can also act on behalf of  several non-EU controllers and processors.

When the function of representative is assumed by a company or any other type of organisation, it is  recommended that a single individual be assigned as a lead contact and person "in charge" for each  controller or processor represented. It would generally also be useful to specify these points in the  service contract.

In line with the GDPR, the EDPB confirms that, when several processing activities of a controller or  processor fall within the scope of Article 3(2) GDPR (and none of the exceptions of Article 27(2) GDPR  apply), that controller or processor is not expected to designate several representatives for each  separate processing activity falling within the scope of article 3(2).The EDPB does not consider the  function of representative in the Union as compatible with the role of an external data protection  officer ("DPO") which would be established in the Union. Article 38(3) establishes some basic  guarantees to help ensure that DPOs are able to perform their tasks with a sufficient degree of  autonomy within their organisation. In particular, controllers or processors are required to ensure that  the DPO "does not receive any instructions regarding the exercise of [his or her] tasks". Recital 97 adds  that DPOs, "whether or not they are an employee of the controller, should be in a position to perform  their duties and tasks in an independent manner"[^28]. Such requirement for a sufficient degree of  autonomy and independence of a data protection officer does not appear to be compatible with the  function of representative in the Union. The representative is indeed subject to a mandate by a  controller or processor and will be acting on its behalf and therefore under its direct instruction[^29]. The  representative is mandated by the controller or processor it represents, and therefore acting on its  behalf in exercising its task, and such a role cannot be compatible with the carrying out of duties and  tasks of the data protection officer in an independent manner.

Furthermore, and to complement its interpretation, the EDPB recalls the position already taken by the  WP29 stressing that "a conflict of interests may also arise for example if an external DPO is asked to  represent the controller or processor before the Courts in cases involving data protection issues"[^30].

Similarly, given the possible conflict of obligation and interests in cases of enforcement proceedings,  the EDPB does not consider the function of a data controller representative in the Union as compatible with the role of data processor for that same data controller, in particular when it comes to compliance  with their respective responsibilities and compliance.

While the GDPR does not impose any obligation on the data controller or the representative itself to  notify the designation of the latter to a supervisory authority, the EDPB recalls that, in accordance with  Articles 13(1)a and 14(1)a, as part of their information obligations, controllers shall provide data  subjects information as to the identity of their representative in the Union. This information shall for  example be included in the [privacy notice and] upfront information provided to data subjects at the  moment of data collection. A controller not established in the Union but falling under Article 3(2) and  failing to inform data subjects who are in the Union of the identity of its representative would be in  breach of its transparency obligations as per the GDPR. Such information should furthermore be easily  accessible to supervisory authorities in order to facilitate the establishment of a contact for  cooperation needs.

Example 24: The website referred to in example 12, based and managed in Turkey, offers services for  the creation, edition, printing and shipping of personalised family photo albums. The website is  available in English, French, Dutch and German and payments can be made in Euros or Sterling. The  website indicates that photo albums can only be delivered by post mail in the France, Benelux  countries and Germany. This website being subject to the GDPR, as per its Article 3(2)(a), the data  controller must designate a representative in the Union.  <br>

The representative must be established in one of the Member States where the service offered is  available, in this case either in France, Belgium, Netherlands, Luxembourg or Germany. The name and  contact details of the data controller and its representative in the Union must be part of the  information made available online to data subjects once they start using the service by creating their  photo album. It must also appear in the website general privacy notice.

  
[^28]: WP29 Guidelines on Data Protection Officers ('DPOs'), WP 243 rev.01 - endorsed by the EDPB.  
[^29]: An external DPO also acting as representative in the Union could not for example be in a situation where he is  instructed, as a representative, to communicate to a data subject a decision or measure taken by the controller  or processor which he or she, as a DPO, had deemed uncompliant with the provisions of the GDPR and advised  against.  
[^30]: WP29 Guidelines on Data Protection Officers ('DPOs'), WP 243 rev.01 - endorsed by the EDPB.